**Hypothesis Testing using different methods**

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
sns.set()

In [ ]:
import pandas as pd
from scipy import stats

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


REading the file

In [ ]:
data = pd.read_csv('/content/drive/MyDrive/capstone_final_preprocessed_data.csv')

In [ ]:
data=data.drop('Unnamed: 0',axis=1)

In [ ]:
data.head()

,Permit Number,Permit Type,Permit Type Definition,Permit Creation Date,Block,Lot,Street Number,Street Name,Street Suffix,Description,Current Status,Current Status Date,Filed Date,Issued Date,First Construction Document Date,Number of Existing Stories,Number of Proposed Stories,Permit Expiration Date,Estimated Cost,Revised Cost,Existing Use,Existing Units,Proposed Use,Proposed Units,Plansets,Existing Construction Type,Existing Construction Type Description,Proposed Construction Type,Proposed Construction Type Description,Supervisor District,Neighborhoods - Analysis Boundaries,Zipcode,Location,Record ID
0,201505065519,4,sign__erect,05/06/2015,0326,023,140,Ellis,St,"ground fl facade: to erect illuminated, electr...",expired,12/21/2017,05/06/2015,11/09/2015,11/09/2015,6.0,6.0,11/03/2016,4000.0,4000.0,tourist_hotel/motel,143.0,tourist_hotel_motel,143.0,2.0,3.0,constr_type_3,3.0,constr_type_3,3.0,tenderloin,94102.0,"(37.785719256680785, -122.40852313194863)",1380611233945
1,201604195146,4,sign__erect,04/19/2016,0306,007,440,Geary,St,remove (e) awning and associated signs.,issued,08/03/2017,04/19/2016,08/03/2017,08/03/2017,7.0,7.0,12/03/2017,1.0,500.0,tourist_hotel/motel,143.0,tourist_hotel_motel,143.0,2.0,3.0,constr_type_3,3.0,constr_type_3,3.0,tenderloin,94102.0,"(37.78733980600732, -122.41063199757738)",1420164406718
2,201611072166,8,otc_alterations_permit,11/07/2016,0156,011,1230,Pacific,Av,repair dryrot & stucco at front of bldg.,complete,07/24/2017,11/07/2016,07/18/2017,07/18/2017,2.0,2.0,07/13/2018,2000.0,2000.0,1_family_dwelling,1.0,1_family_dwelling,1.0,2.0,5.0,wood_frame_5,5.0,wood_frame_5,3.0,nob_hill,94109.0,"(37.79595867909168, -122.41557405519474)",1443574295566
3,201611283529,6,demolitions,11/28/2016,0342,001,950,Market,St,demolish retail/office/commercial 3-story buil...,issued,12/01/2017,11/28/2016,12/01/2017,11/20/2017,3.0,3.0,12/01/2018,100000.0,100000.0,retail_sales,1.0,retail_sales,1.0,2.0,3.0,constr_type_3,3.0,constr_type_3,6.0,tenderloin,94102.0,"(37.78315261897309, -122.40950883997789)",144548169992
4,201706149344,8,otc_alterations_permit,06/14/2017,4105,009,800,Indiana,St,evac maps,issued,07/06/2017,06/14/2017,07/06/2017,07/06/2017,5.0,5.0,07/01/2018,4000.0,4000.0,apartments,326.0,apartments,326.0,2.0,1.0,constr_type_1,1.0,constr_type_1,10.0,potrero_hill,94107.0,"(37.75922331346539, -122.39170402628598)",1466911170855


In [ ]:
data['Permit Creation Date'] =  pd.to_datetime(data['Permit Creation Date'], errors='coerce')

data['Current Status Date'] =  pd.to_datetime(data['Current Status Date'], errors='coerce')

data['Filed Date'] =  pd.to_datetime(data['Filed Date'], errors='coerce')

data['Issued Date'] =  pd.to_datetime(data['Issued Date'], errors='coerce')

data['First Construction Document Date'] =  pd.to_datetime(data['First Construction Document Date'], errors='coerce')

data['Permit Expiration Date'] =  pd.to_datetime(data['Permit Expiration Date'], errors='coerce')

In [ ]:
data.dtypes[:]

Permit Number                                     object
Permit Type                                        int64
Permit Type Definition                            object
Permit Creation Date                      datetime64[ns]
Block                                             object
Lot                                               object
Street Number                                      int64
Street Name                                       object
Street Suffix                                     object
Description                                       object
Current Status                                    object
Current Status Date                       datetime64[ns]
Filed Date                                datetime64[ns]
Issued Date                               datetime64[ns]
First Construction Document Date          datetime64[ns]
Number of Existing Stories                       float64
Number of Proposed Stories                       float64
Permit Expiration Date         

**Defining all the numerical columns**

In [ ]:
cols = data.select_dtypes([np.int64,np.float64]).columns
aa=list(cols)
aa

['Permit Type',
 'Street Number',
 'Number of Existing Stories',
 'Number of Proposed Stories',
 'Estimated Cost',
 'Revised Cost',
 'Existing Units',
 'Proposed Units',
 'Plansets',
 'Existing Construction Type',
 'Proposed Construction Type',
 'Supervisor District',
 'Zipcode',
 'Record ID']

**Defining all the categorical features**

In [ ]:
c = (data.dtypes == 'object')
catcol = list(c[c].index)
catcol

['Permit Number',
 'Permit Type Definition',
 'Block',
 'Lot',
 'Street Name',
 'Street Suffix',
 'Description',
 'Current Status',
 'Existing Use',
 'Proposed Use',
 'Existing Construction Type Description',
 'Proposed Construction Type Description',
 'Neighborhoods - Analysis Boundaries',
 'Location']

In [ ]:
data['Existing Construction Type Description'].unique()

array(['constr_type_3', 'wood_frame_5', 'constr_type_1', 'constr_type_4',
       'constr_type_2'], dtype=object)

**Does Existing Construction Type Description have any significance with any other field in the dataset?**

**Chi-Squared Test** (category vs category)

In [ ]:
for i in catcol:
  print(i)
  contingency_table=pd.crosstab(data['Existing Construction Type Description'],data[i])
  print('contingency_table :-\n',contingency_table)
#Observed Values
  Observed_Values = contingency_table.values 
  print("Observed Values :-\n",Observed_Values)
  b=stats.chi2_contingency(contingency_table)
  Expected_Values = b[3]
  print("Expected Values :-\n",Expected_Values)
  no_of_rows=len(contingency_table.iloc[0:2,0])
  no_of_columns=len(contingency_table.iloc[0,0:2])
  ddof=(no_of_rows-1)*(no_of_columns-1)
  print("Degree of Freedom:-",ddof)
  alpha = 0.05
  from scipy.stats import chi2
  chi_square=sum([(o-e)**2./e for o,e in zip(Observed_Values,Expected_Values)])
  chi_square_statistic=chi_square[0]+chi_square[1]
  print("chi-square statistic:-",chi_square_statistic)
  critical_value=chi2.ppf(q=1-alpha,df=ddof)
  print('critical_value:',critical_value)
#p-value
  p_value=1-chi2.cdf(x=chi_square_statistic,df=ddof)
  print('p-value:',p_value)
  print('Significance level: ',alpha)
  print('Degree of Freedom: ',ddof)
  print('chi-square statistic:',chi_square_statistic)
  print('critical_value:',critical_value)
  print('p-value:',p_value)
  if chi_square_statistic>=critical_value:
    print("Reject H0,There is a relationship between 2 categorical variables")
  else:
    print("Retain H0,There is no relationship between 2 categorical variables")
    
  if p_value<=alpha:
    print("Reject H0,There is a relationship between 2 categorical variables")
  else:print("Retain H0,There is no relationship between 2 categorical variables")
  print('*'*50)

Permit Number
contingency_table :-
 Permit Number                           201301027037  ...  M893427
Existing Construction Type Description                ...         
constr_type_1                                      1  ...        0
constr_type_2                                      0  ...        0
constr_type_3                                      0  ...        0
constr_type_4                                      0  ...        0
wood_frame_5                                       0  ...        1

[5 rows x 166686 columns]
Observed Values :-
 [[1 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 1 1 ... 1 1 1]]
Expected Values :-
 [[0.18509151 0.18509151 0.18509151 ... 0.18509151 0.18509151 0.18509151]
 [0.0262026  0.0262026  0.0262026  ... 0.0262026  0.0262026  0.0262026 ]
 [0.06203915 0.06203915 0.06203915 ... 0.06203915 0.06203915 0.06203915]
 [0.00244689 0.00244689 0.00244689 ... 0.00244689 0.00244689 0.00244689]
 [0.72421985 0.72421985 0.72421985 ... 0

**Is there any significant relationship of Existing Construction Type Description with any other feature in the dataset, if yes then what all are the features?**

**Annova test**  (category vs numerical)

In [ ]:
  df_anova = data
  print('Permit Type')
  df_anova = df_anova[['Existing Construction Type Description','Permit Type']]
  grps = pd.unique(df_anova['Existing Construction Type Description'].values)
  d_data = {grp:df_anova['Permit Type'][df_anova['Existing Construction Type Description'] == grp] for grp in grps}
 
  F, p = stats.f_oneway(d_data['constr_type_3'], d_data['wood_frame_5'], d_data['constr_type_1'], d_data['constr_type_4'], d_data['constr_type_2'])
  print("p-value for significance is: ", p)
  if p<0.05:
    print("reject null hypothesis")
  else:
    print("accept null hypothesis")

Permit Type
p-value for significance is:  4.0823063258032324e-76
reject null hypothesis


In [ ]:
  df_anova1 = data
  print('Street Number')
  df_anova1 = df_anova1[['Existing Construction Type Description','Street Number']]
  grps = pd.unique(df_anova1['Existing Construction Type Description'].values)
  d_data1 = {grp:df_anova1['Street Number'][df_anova1['Existing Construction Type Description'] == grp] for grp in grps}
 
  F, p = stats.f_oneway(d_data1['constr_type_3'], d_data1['wood_frame_5'], d_data1['constr_type_1'], d_data1['constr_type_4'], d_data1['constr_type_2'])
  print("p-value for significance is: ", p)
  if p<0.05:
    print("reject null hypothesis")
  else:
    print("accept null hypothesis")

Street Number
p-value for significance is:  0.0
reject null hypothesis


In [ ]:
  df_anova1 = data
  print('Number of Existing Stories')
  df_anova1 = df_anova1[['Existing Construction Type Description','Number of Existing Stories']]
  grps = pd.unique(df_anova1['Existing Construction Type Description'].values)
  d_data1 = {grp:df_anova1['Number of Existing Stories'][df_anova1['Existing Construction Type Description'] == grp] for grp in grps}
 
  F, p = stats.f_oneway(d_data1['constr_type_3'], d_data1['wood_frame_5'], d_data1['constr_type_1'], d_data1['constr_type_4'], d_data1['constr_type_2'])
  print("p-value for significance is: ", p)
  if p<0.05:
    print("reject null hypothesis")
  else:
    print("accept null hypothesis")

Number of Existing Stories
p-value for significance is:  0.0
reject null hypothesis


In [ ]:
  df_anova1 = data
  print('Number of Proposed Stories')
  df_anova1 = df_anova1[['Existing Construction Type Description','Number of Proposed Stories']]
  grps = pd.unique(df_anova1['Existing Construction Type Description'].values)
  d_data1 = {grp:df_anova1['Number of Proposed Stories'][df_anova1['Existing Construction Type Description'] == grp] for grp in grps}
 
  F, p = stats.f_oneway(d_data1['constr_type_3'], d_data1['wood_frame_5'], d_data1['constr_type_1'], d_data1['constr_type_4'], d_data1['constr_type_2'])
  print("p-value for significance is: ", p)
  if p<0.05:
    print("reject null hypothesis")
  else:
    print("accept null hypothesis")

Number of Proposed Stories
p-value for significance is:  0.0
reject null hypothesis


In [ ]:
  df_anova1 = data
  print('Estimated Cost')
  df_anova1 = df_anova1[['Existing Construction Type Description','Estimated Cost']]
  grps = pd.unique(df_anova1['Existing Construction Type Description'].values)
  d_data1 = {grp:df_anova1['Estimated Cost'][df_anova1['Existing Construction Type Description'] == grp] for grp in grps}
 
  F, p = stats.f_oneway(d_data1['constr_type_3'], d_data1['wood_frame_5'], d_data1['constr_type_1'], d_data1['constr_type_4'], d_data1['constr_type_2'])
  print("p-value for significance is: ", p)
  if p<0.05:
    print("reject null hypothesis")
  else:
    print("accept null hypothesis")

Estimated Cost
p-value for significance is:  3.354035704406078e-69
reject null hypothesis


In [ ]:
  df_anova1 = data
  print('Revised Cost')
  df_anova1 = df_anova1[['Existing Construction Type Description','Revised Cost']]
  grps = pd.unique(df_anova1['Existing Construction Type Description'].values)
  d_data1 = {grp:df_anova1['Revised Cost'][df_anova1['Existing Construction Type Description'] == grp] for grp in grps}
 
  F, p = stats.f_oneway(d_data1['constr_type_3'], d_data1['wood_frame_5'], d_data1['constr_type_1'], d_data1['constr_type_4'], d_data1['constr_type_2'])
  print("p-value for significance is: ", p)
  if p<0.05:
    print("reject null hypothesis")
  else:
    print("accept null hypothesis")

Revised Cost
p-value for significance is:  7.178799546001898e-68
reject null hypothesis


In [ ]:
  df_anova1 = data
  print('Existing Units')
  df_anova1 = df_anova1[['Existing Construction Type Description','Existing Units']]
  grps = pd.unique(df_anova1['Existing Construction Type Description'].values)
  d_data1 = {grp:df_anova1['Existing Units'][df_anova1['Existing Construction Type Description'] == grp] for grp in grps}
 
  F, p = stats.f_oneway(d_data1['constr_type_3'], d_data1['wood_frame_5'], d_data1['constr_type_1'], d_data1['constr_type_4'], d_data1['constr_type_2'])
  print("p-value for significance is: ", p)
  if p<0.05:
    print("reject null hypothesis")
  else:
    print("accept null hypothesis")

Existing Units
p-value for significance is:  0.0
reject null hypothesis


In [ ]:
  df_anova1 = data
  print('Proposed Units')
  df_anova1 = df_anova1[['Existing Construction Type Description','Proposed Units']]
  grps = pd.unique(df_anova1['Existing Construction Type Description'].values)
  d_data1 = {grp:df_anova1['Proposed Units'][df_anova1['Existing Construction Type Description'] == grp] for grp in grps}
 
  F, p = stats.f_oneway(d_data1['constr_type_3'], d_data1['wood_frame_5'], d_data1['constr_type_1'], d_data1['constr_type_4'], d_data1['constr_type_2'])
  print("p-value for significance is: ", p)
  if p<0.05:
    print("reject null hypothesis")
  else:
    print("accept null hypothesis")

Proposed Units
p-value for significance is:  0.0
reject null hypothesis


In [ ]:
  df_anova1 = data
  print('Plansets')
  df_anova1 = df_anova1[['Existing Construction Type Description','Plansets']]
  grps = pd.unique(df_anova1['Existing Construction Type Description'].values)
  d_data1 = {grp:df_anova1['Plansets'][df_anova1['Existing Construction Type Description'] == grp] for grp in grps}
 
  F, p = stats.f_oneway(d_data1['constr_type_3'], d_data1['wood_frame_5'], d_data1['constr_type_1'], d_data1['constr_type_4'], d_data1['constr_type_2'])
  print("p-value for significance is: ", p)
  if p<0.05:
    print("reject null hypothesis")
  else:
    print("accept null hypothesis")

Plansets
p-value for significance is:  2.5656957222716467e-09
reject null hypothesis


In [ ]:
  df_anova1 = data
  print('Existing Construction Type')
  df_anova1 = df_anova1[['Existing Construction Type Description','Existing Construction Type']]
  grps = pd.unique(df_anova1['Existing Construction Type Description'].values)
  d_data1 = {grp:df_anova1['Existing Construction Type'][df_anova1['Existing Construction Type Description'] == grp] for grp in grps}
 
  F, p = stats.f_oneway(d_data1['constr_type_3'], d_data1['wood_frame_5'], d_data1['constr_type_1'], d_data1['constr_type_4'], d_data1['constr_type_2'])
  print("p-value for significance is: ", p)
  if p<0.05:
    print("reject null hypothesis")
  else:
    print("accept null hypothesis")

Existing Construction Type
p-value for significance is:  0.0
reject null hypothesis


In [ ]:
  df_anova1 = data
  print('Proposed Construction Type')
  df_anova1 = df_anova1[['Existing Construction Type Description','Proposed Construction Type']]
  grps = pd.unique(df_anova1['Existing Construction Type Description'].values)
  d_data1 = {grp:df_anova1['Proposed Construction Type'][df_anova1['Existing Construction Type Description'] == grp] for grp in grps}
 
  F, p = stats.f_oneway(d_data1['constr_type_3'], d_data1['wood_frame_5'], d_data1['constr_type_1'], d_data1['constr_type_4'], d_data1['constr_type_2'])
  print("p-value for significance is: ", p)
  if p<0.05:
    print("reject null hypothesis")
  else:
    print("accept null hypothesis")

Proposed Construction Type
p-value for significance is:  0.0
reject null hypothesis


In [ ]:
  df_anova1 = data
  print('Supervisor District')
  df_anova1 = df_anova1[['Existing Construction Type Description','Supervisor District']]
  grps = pd.unique(df_anova1['Existing Construction Type Description'].values)
  d_data1 = {grp:df_anova1['Supervisor District'][df_anova1['Existing Construction Type Description'] == grp] for grp in grps}
 
  F, p = stats.f_oneway(d_data1['constr_type_3'], d_data1['wood_frame_5'], d_data1['constr_type_1'], d_data1['constr_type_4'], d_data1['constr_type_2'])
  print("p-value for significance is: ", p)
  if p<0.05:
    print("reject null hypothesis")
  else:
    print("accept null hypothesis")

Supervisor District
p-value for significance is:  0.0
reject null hypothesis


In [ ]:
  df_anova1 = data
  print('Zipcode')
  df_anova1 = df_anova1[['Existing Construction Type Description','Zipcode']]
  grps = pd.unique(df_anova1['Existing Construction Type Description'].values)
  d_data1 = {grp:df_anova1['Zipcode'][df_anova1['Existing Construction Type Description'] == grp] for grp in grps}
 
  F, p = stats.f_oneway(d_data1['constr_type_3'], d_data1['wood_frame_5'], d_data1['constr_type_1'], d_data1['constr_type_4'], d_data1['constr_type_2'])
  print("p-value for significance is: ", p)
  if p<0.05:
    print("reject null hypothesis")
  else:
    print("accept null hypothesis")

Zipcode
p-value for significance is:  0.0
reject null hypothesis


In [ ]:
df_anova1 = data
print('Record ID')
df_anova1 = df_anova1[['Existing Construction Type Description','Record ID']]
grps = pd.unique(df_anova1['Existing Construction Type Description'].values)
d_data1 = {grp:df_anova1['Record ID'][df_anova1['Existing Construction Type Description'] == grp] for grp in grps}
F, p = stats.f_oneway(d_data1['constr_type_3'], d_data1['wood_frame_5'], d_data1['constr_type_1'], d_data1['constr_type_4'], d_data1['constr_type_2'])
print("p-value for significance is: ", p)
if p<0.05:
  print("reject null hypothesis")
else:
  print("accept null hypothesis")

Record ID
p-value for significance is:  0.0
reject null hypothesis


Other test to verify

Pearson correlation

In [ ]:
from scipy.stats import pearsonr
data1 = data['Existing Construction Type']
data2 = data['Permit Type']
stat, p = pearsonr(data1, data2)
print('stat=%.3f, p=%.3f' % (stat, p))
if p > 0.05:
	print('Probably independent')
else:
	print('Probably dependent')

stat=0.021, p=0.000
Probably dependent


Mann-Whitney U Test

In [ ]:
from scipy.stats import mannwhitneyu
data1 = data['Existing Construction Type']
data2 = data['Permit Type']
stat, p = mannwhitneyu(data1, data2)
print('stat=%.3f, p=%.3f' % (stat, p))
if p > 0.05:
	print('Probably the same distribution')
else:
	print('Probably different distributions')

stat=1732764707.000, p=0.000
Probably different distributions
